# Estimating true symptomatic infections -- timeseries

Note: see code for details and also links/references (embedded as comments).

We want to estimate the number of true symptomatic infections, because we believe that the reported cases are an underestimate (due to imperfect reporting rates). We restrict our analysis to symptomatic infections because asymptomatic infections do not spread measles.

Suppose the true/total number of log-symptomatic infections follows a random walk -- this means we have smooth exponential growth:
$$\log I_{t+1} = \log I_t + \epsilon_t, \quad \epsilon_t \sim \mathcal N(0, \sigma^2)$$

Let $\rho$ be the reporting rate. Note that we have data on REPORTED cases $C^\text{reported}_t$ from the linelist data:
$$C^{\text{reported}}_t \sim \text{Poisson}(I_t \cdot  \rho)$$ 

For hospitalizations and deaths, we have
$$H_t \sim \text{Poisson}(I_{t-\delta_0} \cdot \widehat{\text{IHR}}),$$
where $\widehat{\text{IHR}}$ is the estimated (symptomatic) infection hospitalization rate and $\delta_0$ is the time from displaying symptoms to getting hospitalized (in days), and

$$D_t \sim \text{Poisson}(I_{t-\delta_1} \cdot \text{IFR}),$$
where $\text{IFR}$ is the infection fatality rate and $\delta_1$ is the time that a person spends in the hospital before death (in days).


### Modeling notes: priors

- We assume $\sigma^2 = (0.1)^2$ in the random walk parametrization.
- We set the prior for the reporting rate $\rho$ to be Uniform in $[0.1, 0.9]$.
- We set the prior for IFR to be Uniform in $[1/1000, 3/1000]$ (taken from the literature).

### Modeling notes: IHR

We could not find literature on IHR, so we did our best approximation. 

The CDC reports that 1 in 20 unvaccinated people who get measles are hospitalized. 

We do not know the proportion infected who are unvaccinated, but we know the proportion of cases that are unvaccinated. If we assume that the reporting rate is the same regardless of vaccination status (which might not actually be true), then the proportion infected who are unvaccinated equals the proportion of cases that are unvaccinated.

The correct formula for IHR is $$IHR = \texttt{IHR-unvax} \times \texttt{prop infected unvax} + \texttt{IHR-vax} \times \texttt{prop  infected vax},$$
where $\texttt{IHR-unvax}$ is the infection hospitalization rate for unvaccinated people and $\texttt{prop infected unvax}$ is the proportion of infected people who are unvaccinated, and analogously for $\texttt{IHR-vax}$ and $\texttt{prop infected vax}$.

We assume that $\texttt{prop cases unvax} \approx \texttt{prop infected unvax}$ and $\texttt{IHR-vax} \approx 0$, so we use 
$$\widehat{\texttt{IHR}} = \texttt{IHR-unvax} \times \texttt{prop infected unvax}$$ which comes out to be about $0.048.$ 

### Public health input needed: delay times

We have $\delta_0 = 5$ (the time from symptom onset until hospitalization). This is taken from a CDC source that says rashes break out 3 to 5 days after symptom onset, so we are roughly positing that hospitalization happens around the same time the rash breaks out.
 
We have $\delta_1 = 7$ (the time from entering the hospital to death), which is a total total guess. We'll need public health input on this.

### Technical note: Poisson vs Binomial distribution

Lauren brought up a good question about why we would use Poisson versus Binomial, or vice versa. 

I (LP) am choosing to use Poisson instead of Binomial (for all relevant variables) due to some modeling and coding issues. 
- First, the Poisson rate does not need to be integer. We need this relaxation in order to use a random walk for $\log I_t$ -- because in our model, the sampled $I_t$ is continuous.
- Second, the Binomial distribution does not work well with shifted time series. The shifts are due to the delays $\delta_0$ and $\delta_1$. This is because, for example for $H_t$, if the historical number of hospital admits at time $t$ (from the linelist data) is greater than $I_{t-\delta_0}$ (which gets passed to the "$n$" parameter in a Binomial distribution), then we cannot compute the likelihood. It's impossible for a Binomial random variable with number of trials $n$ to generate a number of successes $s$ that is greater than $n$. So, the code breaks. This is a pretty rigid requirement when working with shifted time series, but we don't have this problem with the Poisson distribution.





# Estimating true symptomatic infections -- total

This was suggested by Remy. Instead of looking at the time series, we could apply a much simpler hierarchal Bayesian model to total counts. Note that we don't have the same Poisson issues as before, so here we just use Binomial distributions. The formulation is as follows. 

Let $\rho$ be the reporting rate. Note that we have data on REPORTED cases $C^\text{reported}$ from the linelist data:
$$C^{\text{reported}} \sim \text{Poisson}(I \cdot  \rho),$$

where $C^\text{reported}$ is the \textit{total} number of reported cases over the linelist time period, and $I$ is the \textit{total} number of symptomatic infections over this same period. Both are scalars.

For \textit{total} hospitalizations $H$ and \textit{total} deaths $D$, we have
$$H \sim \text{Binomial}(I, \widehat{\text{IHR}}),$$
where $\widehat{\text{IHR}}$ is the estimated (symptomatic) infection hospitalization rate defined previously and

$$D \sim \text{Poisson}(I, \cdot \text{IFR}).$$



# Computational implementation

We used `pyMC` to code up these hierarchical Bayesian models and used the NUTS (No U-Turn Sampler), which is a Hamiltonian Monte Carlo sampler. 